In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

import librosa
import os as os
from scipy.misc import comb
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
from tqdm import tqdm
from keras.models import load_model

C:\Users\Blaine\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
DATA_PATH = "./large_files/audio/"  


def get_labels(path=DATA_PATH):
    labels = os.listdir(path)
    label_indices = np.arange(0, len(labels))
    return labels, label_indices, to_categorical(label_indices)
# Get available labels
labels, indices, _ = get_labels(DATA_PATH)


# Getting first arrays
X = np.load(labels[0] + '.npy')
y = np.zeros(X.shape[0])

# Append all of the dataset into one single array, same goes for y
for i, label in enumerate(labels[1:]):
    x = np.load(label + '.npy')
    X = np.vstack((X, x))
    y = np.append(y, np.full(x.shape[0], fill_value= (i + 1)))

assert X.shape[0] == len(y)

In [3]:
%reload_ext autoreload
%autoreload 2

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.utils import to_categorical


# Second dimension of the feature is dim2
feature_dim_2 = 11

# # Feature dimension
feature_dim_1 = 20
channel = 1
epochs = 10
batch_size = 50
verbose = 2
num_classes = 30

# Reshaping to perform 2D convolution
X = X.reshape(X.shape[0], feature_dim_1, feature_dim_2, channel)


y_hot = to_categorical(y)

In [5]:
def get_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(2, 2), activation='relu', input_shape=(feature_dim_1, feature_dim_2, channel)))
    model.add(Conv2D(48, kernel_size=(2, 2), activation='relu'))
    model.add(Conv2D(120, kernel_size=(2, 2), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

# Predicts one sample
def predict(filepath, model):
    sample = wav2mfcc(filepath)
    sample_reshaped = sample.reshape(1, feature_dim_1, feature_dim_2, channel)
    return get_labels()[0][
            np.argmax(model.predict(sample_reshaped))
    ]

In [6]:
model = get_model()
model.fit(X, y_hot,
          batch_size=batch_size, 
          epochs=epochs, 
          verbose=verbose
         )

Epoch 1/50
 - 23s - loss: 2.8926 - acc: 0.1671
Epoch 2/50
 - 9s - loss: 1.6248 - acc: 0.5163
Epoch 3/50
 - 9s - loss: 1.1808 - acc: 0.6556
Epoch 4/50
 - 9s - loss: 0.9945 - acc: 0.7133
Epoch 5/50
 - 9s - loss: 0.8951 - acc: 0.7434
Epoch 6/50
 - 9s - loss: 0.8214 - acc: 0.7662
Epoch 7/50
 - 9s - loss: 0.7725 - acc: 0.7786
Epoch 8/50
 - 9s - loss: 0.7321 - acc: 0.7907
Epoch 9/50
 - 9s - loss: 0.6998 - acc: 0.8010
Epoch 10/50
 - 9s - loss: 0.6741 - acc: 0.8084
Epoch 11/50
 - 9s - loss: 0.6559 - acc: 0.8141
Epoch 12/50
 - 9s - loss: 0.6448 - acc: 0.8181
Epoch 13/50
 - 9s - loss: 0.6355 - acc: 0.8215
Epoch 14/50
 - 9s - loss: 0.6235 - acc: 0.8279
Epoch 15/50
 - 9s - loss: 0.6138 - acc: 0.8291
Epoch 16/50
 - 9s - loss: 0.6144 - acc: 0.8305
Epoch 17/50
 - 9s - loss: 0.6062 - acc: 0.8322
Epoch 18/50
 - 9s - loss: 0.6026 - acc: 0.8337
Epoch 19/50
 - 10s - loss: 0.5955 - acc: 0.8362
Epoch 20/50
 - 9s - loss: 0.5872 - acc: 0.8373
Epoch 21/50
 - 9s - loss: 0.5860 - acc: 0.8393
Epoch 22/50
 - 9s - 